In [3]:
import pandas as pd
import numpy as np
import torch
import logging
from sklearn.model_selection import train_test_split

from datasets import Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score

In [5]:
# Load CSV
df = pd.read_csv('Data/processed_ai_vs_human.csv')
df.head()

,text,generated
0,Cars Cars around since became famous 1900s Hen...,0
1,Transportation large necessity countries world...,0
2,Americas love affair vehicles seems cooling sa...,0
3,often ride car drive one motor vehicle work st...,0
4,Cars wonderful thing perhaps one worlds greate...,0


In [6]:
# Split dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [7]:
# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize
def tokenize_function(examples):
        try:
            return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
        except Exception as e:
            return {}

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

tokenized_train.set_format("torch")
tokenized_test.set_format("torch")

IndentationError: unindent does not match any outer indentation level (<string>, line 5)

In [ ]:
# Training args
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

In [ ]:
# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [ ]:
# Trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [ ]:
# Train
trainer.train()

In [ ]:
# Evaluate
trainer.evaluate()

In [ ]:
# Save model
model.save_pretrained("./results/final_model")
tokenizer.save_pretrained("./results/final_model")